In [12]:
# import libraries for web scraping
import requests
from bs4 import BeautifulSoup
import selenium as se
import pandas as pd
import numpy as np
import time
import os

base_player_url = 'https://www.pro-football-reference.com'
player_urls_filtered = pd.read_csv('scraping/player_urls_filtered.csv')['player_url']

In [ ]:
# load StatHead player ids
stathead = pd.read_csv('scraping/stat-head_player_data.csv')
template_url = 'https://www.pro-football-reference.com/players/'

# split the player names
stathead[['first_name', 'last_name']] = stathead['Player'].str.split(' ', n=1, expand=True)

# save the player page urls
player_urls = []
for last_name, id in zip(stathead['last_name'], stathead['PlayerId']):
    player_urls.append(template_url + last_name[0] + '/' + id + '.htm')

# get the tables
pd.Series(player_urls).to_csv('scraping/player_urls.csv', index=False)

For simplicity, we will only retrieve data on the top fantasy players from each season. Additionally, we will save on the number of read operations by limiting the data we are trying to retrieve for each player. For example, we won't try to retrieve receiving stats for quarterbacks or passing stats for running backs. 

Currently, we only have player IDs for players active in 2023. We need all player IDs. 

In [ ]:
import requests
from bs4 import BeautifulSoup
import selenium as se
import pandas as pd
import numpy as np
import time

base_url = 'https://www.pro-football-reference.com/players/'
target_section_id = 'div_players'
outer_element = 'p'
inner_element = 'a'
target_attribute = 'href'

letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
letters = np.random.permutation(list(letters))
player_info = pd.DataFrame(columns=['player_name', 'player_url'])

# proxies = pd.read_csv('proxies.csv')
# proxies = np.random.permutation(proxies['proxy'])
# i = 1

for letter in letters:
    url = base_url + letter
    print(url)
    # print(f'Attempting to use {proxies[i]}')
    page = requests.get(url)
    # while page.status_code != 200 and i < len(proxies):
    #     i += 1
    #     page = requests.get(url, proxies={"http": proxies[i], "https": proxies[i]})
    soup = BeautifulSoup(page.content, 'html.parser')
    target_section = soup.find(id=target_section_id)
    outer_elements = target_section.find_all(outer_element)
    inner_elements = [element.find(inner_element) for element in outer_elements]

    # this doesn't work as desired; results in list of lists
    player_url = [element.get(target_attribute) for element in inner_elements]
    player_name = [element.get_text() for element in inner_elements]
    player_info = pd.concat([pd.DataFrame([[player_name, player_url]], columns=player_info.columns), player_info], ignore_index=True)
    wait = np.random.randint(10, 15)
    time.sleep(wait)

# convert list of lists into dataframe and save
tmp = player_info.copy()
names = pd.Series()
urls = pd.Series()
for name, url in zip(tmp['player_name'], tmp['player_url']):
    names = pd.concat([names, pd.Series(name)], ignore_index=True)
    urls = pd.concat([urls, pd.Series(url)], ignore_index=True)

pd.DataFrame({'player_name': names, 'player_url': urls}).to_csv('player_urls_all.csv', index=False)


In [18]:
# load the player urls
# get the pages
# get the relevant tables
player_urls = pd.read_csv('player_urls_all.csv')
example = player_urls[player_urls['player_name'] == 'Josh Allen']


From example it is clear that there are duplicates that we will need to handle. The first thing we can do is remove players that don't play the positions of interest (QB, RB, WR, TE) from player_urls_all and save to player_urls_filtered. 

In [ ]:
# Let's get the pages for the players
import os
os.chdir('/Users/ryan-saloma/Python Projects/fantasy_football/')
base_player_url = 'https://www.pro-football-reference.com'
player_urls_all = pd.read_csv('scraping/player_urls_all.csv')
player_urls = player_urls_all['player_url']

# Load league data from 1970 to 2023
# Filter urls for players who played in this time frame
# Remove duplicates
# Problem: more than one player can have the same name

# Change the directory to the league data
# Concatenate the data
league_data = pd.DataFrame()
for year in range(1970, 2024):
    file = f'data/league/clean/cleaned_fantasy_{year}.csv'
    data = pd.read_csv(file)
    league_data = pd.concat([league_data, data], ignore_index=True)

# Get the player names
players = league_data.drop_duplicates(subset=['player', 'position'])[['player', 'position']]

# Load the player urls
player_urls = pd.read_csv('scraping/player_urls_all.csv')

# Match player names with player urls
# Save the matched player urls
matched_urls = pd.DataFrame()
for name, position in zip(players['player'], players['position']):
    matched = player_urls[player_urls['player_name'] == name]
    matched['position'] = position
    matched_urls = pd.concat([matched_urls, matched], ignore_index=True)

matched_urls.to_csv('scraping/player_urls_filtered.csv', index=False)

In [ ]:
# make this work in batches to reduce the number of pages stored at any given time
for i in range(0, 906):
    url = player_urls_filtered[i]
    player_page = requests.get(base_player_url + url)
    if player_page.status_code == 200:
        with open('scraping/player_pages/' + url.split('/')[-1], 'w') as f:
            f.write(player_page.text)
    else:
        print(f'Failed to get page for {url}')
    wait = np.random.randint(2, 3)
    time.sleep(wait)

In [16]:
def get_position(page):
    soup = BeautifulSoup(page, 'html.parser')
    # div id="info" > div id="meta" > div > 2nd p tag > text
    position = soup.find('div', {'id': 'info'}).find('div', {'id': 'meta'}).find_all('p')[1].get_text()
    return clean_position_text(position)

def clean_position_text(position):
    position = position.replace('Position:', '').strip()
    # remove "Throws:" and "Right" or "Left"
    if 'Throws:' in position:
        position = position.split('Throws:')[0].strip()
    return position

In [63]:
# Loop through files in player_pages
base_path = 'scraping/player_pages/'
files = [file for file in os.listdir(base_path) if file.endswith('.htm')]
positions = pd.DataFrame(columns=['player_url', 'position'])

for file in files:
    with open(base_path + file, 'r') as f:
        page = f.read()
        position = get_position(page)
        positions = pd.concat([positions, pd.DataFrame([[file, position]], columns=positions.columns)], ignore_index=True)

positions.to_csv('scraping/player_urls_with_positions.csv', index=False)

In [13]:
def batch_process_player_pages(start, end):
    base_path = 'scraping/player_pages/'
    base_player_url = 'https://www.pro-football-reference.com'
    player_urls_filtered = pd.read_csv('scraping/player_urls_filtered.csv')['player_url']
    for i in range(start, end):
        url = player_urls_filtered[i]
        player_page = requests.get(base_player_url + url)
        if player_page.status_code == 200:
            with open('scraping/player_pages/' + url.split('/')[-1], 'w') as f:
                f.write(player_page.text)
        else:
            print(f'Failed to get page for {url}')
        wait = np.random.randint(2, 3)
        time.sleep(wait)

    # Loop through files in player_pages
    base_path = 'scraping/player_pages/'
    files = [file for file in os.listdir(base_path) if file.endswith('.htm')]
    positions = pd.DataFrame(columns=['player_url', 'position'])

    for file in files:
        with open(base_path + file, 'r') as f:
            page = f.read()
            position = get_position(page)
            positions = pd.concat([positions, pd.DataFrame([[file, position]], columns=positions.columns)], ignore_index=True)

    positions.to_csv('scraping/player_urls_with_positions.csv', index=False)

    # Load the player urls with positions
    player_positions = pd.read_csv('scraping/player_urls_with_positions.csv')
    player_positions_na = player_positions[player_positions['position'].isna()]
    player_positions = player_positions.dropna()
    player_positions = player_positions[~player_positions['position'].str.contains('QB|RB|WR|TE')]

    # Concatenate player_positions with player_positions_na
    player_positions = pd.concat([player_positions, player_positions_na], ignore_index=True)

    # Get the files to delete
    files = [file for file in os.listdir('scraping/player_pages/') if file.endswith('.htm')]
    files_to_delete = [file for file in files if player_positions['player_url'].str.contains(file).any()]

    for file in files_to_delete:
        print(f'Deleting file: {file}')
        os.remove('scraping/player_pages/' + file)

In [ ]:
wd = os.getcwd()
if wd != '/Users/ryan-saloma/Python Projects/fantasy_football/':
    os.chdir('/Users/ryan-saloma/Python Projects/fantasy_football/')
player_urls_filtered = pd.read_csv('scraping/player_urls_filtered.csv')['player_url']
batch_process_player_pages(0, 7782)

In [ ]:
# # Get names of sections of interest
# sections = pd.read_csv('scraping/sections.csv')

# # Create a dict that maps a key word to a section header

# # Test getting tables from player pages
# base_path = 'scraping/player_pages/'

# # Get the tables for a QB
# player_page_name = 'AlleJo02.htm'
# tables_qb = pd.read_html(base_path + player_page_name)

# # Repeat for a WR and RB 
# player_page_name = 'SmitDe07.htm'
# tables_wr = pd.read_html(base_path + player_page_name)

# player_page_name = 'McCaCh01.htm'
# tables_rb = pd.read_html(base_path + player_page_name)

# # An alternative approach would be to use AI to classify the tables
# dict = {
#     'Date' : 'Game Logs',
#     'AV' : 'Passing, Regular Season',
#     'Passing_IAY' : 'Advanced Passing, Air Yards',
#     'Passing_Bats' : 'Advanced Passing, Accuracy',
#     'Passing_PktTime' : 'Advanced Passing, Pressure',
#     'RPO_Plays' : 'Advanced Passing, Play Type',
#     'Y/A+' : 'Adjusted Passing',
#     'FantPt' : 'Fantasy',
#     'Receiving_Succ%' : 'Rushing and Receiving',
#     'Rushing_BrkTkl' : 'Advanced Rushing and Receiving',
#     # 'Fumbles_Fmb' : 'Defense and Fumbles',
#     # 'Pts/G' : 'Scoring Summary',
#     'Off._Pct' : 'Snap Counts'
# }

# # Function to flatten the MultiIndex
# def flatten_columns(df):
#     # Create a list of flattened column names
#     new_columns = []
#     for col in df.columns:
#         # Check if first level is unnamed
#         if col[0] == '' or 'Unnamed' in col[0]:
#             new_columns.append(col[1])  # Use the second level name
#         elif col[1] == '' or 'Unnamed' in col[1]:
#             next
#         else:
#             new_columns.append(f"{col[0]}_{col[1]}")  # Combine both levels
    
#     df.columns = new_columns  # Set new columns
#     return df

# cols = []
# new_tables = []
# for table in tables_qb:
#     tmp = table
#     # check if the columns are multi-index
#     if isinstance(tmp.columns, pd.MultiIndex):
#         # flatten the columns
#         tmp = flatten_columns(tmp)
#     # concatenate the columns
#     # cols = cols + list(tmp.columns)
#     new_tables.append(tmp)

# i = 1
# for table in new_tables:
#     cols = table.columns
#     # # print table number and contents
#     for key in dict.keys():
#         if key in cols:
#             print(f'Table {i} contains {key} in {dict[key]}')
#     i += 1

In [94]:
# get all of the htm files in the player_pages directory
# get the tables from each file
# create a new folder for each player using their player id (characters before .htm)
# save tables in the folder
# label as 'table_{number}.csv'

# Get the player ids
player_urls = pd.read_csv('scraping/player_urls_filtered.csv')['player_url']
player_ids = player_urls.str.split('/').str[-1].str.replace('.htm', '')

# Get the files in the player_pages directory
base_path = 'scraping/player_pages/'
files = [file for file in os.listdir(base_path) if file.endswith('.htm')]

# Get the tables for each player
for file in files:
    tables = pd.read_html(base_path + file)
    player_id = file.replace('.htm', '')
    player_path = f'scraping/player_tables/{player_id}/'
    if not os.path.exists(player_path):
        os.makedirs(player_path)
    for i, table in enumerate(tables):
        table.to_csv(player_path + f'table_{i}.csv', index=False)

In [96]:
# This block of code extracts the player id from the player url

# Derive column 'player_id' from column: 'player_url'
def player_id(player_url):
    """
    Transform based on the following examples:
       player_url                   Output
    1: "/players/B/BaabMi20.htm" => "BaabMi20"
    """
    index1 = [i for i in range(len(player_url)) if player_url.startswith("/", i)][2] + 1
    return player_url[index1:player_url.find(".")]

# Get dataframe
df = pd.read_csv("scraping/player_urls_all.csv")

# Apply function
df.insert(2, "player_id", df.apply(lambda row : player_id(row["player_url"]), axis=1))

# Save to csv
df.to_csv("scraping/player_urls_all.csv", index=False)

# Repeat for player_urls_filtered.csv
df = pd.read_csv("scraping/player_urls_filtered.csv")
df.insert(2, "player_id", df.apply(lambda row : player_id(row["player_url"]), axis=1))
df.to_csv("scraping/player_urls_filtered.csv", index=False)


In [97]:
# This block of code reformats player_urls_all.csv to stathead_player_ids.csv
# This csv will be used to create database

# Load player_urls_all.csv
df = pd.read_csv("scraping/player_urls_all.csv")

# Remove player_url column
df = df.drop(columns=["player_url"])

# Make the player_id column the first column
df = df[["player_id", "player_name"]]

# Save to csv
df.to_csv("scraping/stathead_player_ids.csv", index=False)